In [ ]:
import pandas as pd
import numpy as np

from data.data_reader import read
from fitting.line_fitter_3d import fit_line, get_line
from fitting.point_line_distance import get_distances
from fitting.vector_geometry_helper import standard_parametric_vector, enclosing_angle
from visualization.cloud_visualization import plot_3d_line, plot_3d_cloud

% matplotlib inline

In [ ]:
df = read(nrows=10000)
df.head()

In [ ]:
print(df['parentID'].unique())
print(len(df))

In [ ]:

df_primary = df[df['parentID'] == 0]
print(len(df_primary))
print(df['eventID'].unique())

MAX_EVENT_ID = df['eventID'].max()
POOL = df['eventID'].unique()

plot_3d_cloud(df)

In [ ]:

# for eventID in POOL:
#     df_tmp = df[df['eventID'] == eventID].copy(deep=True)
#     plot_3d_cloud(df_tmp, 'eventID = ' + str(eventID))
#     message = input('--> ')
#     if message == '-1':
#         break


In [ ]:
def calc_statistical_moments(part):
    points = part[['posX', 'posY', 'posZ']]
    u, v = fit_line(points)
    distances = get_distances(u, v, points)
    part['mean'] = distances.mean()
    part['std'] = distances.std()
    part['var'] = distances.var()
    return part


df_primary.groupby(['eventID']).apply(calc_statistical_moments)

In [ ]:
tracks = df_primary.groupby(['eventID'])

statistic = []
for _, track in tracks:
    points = track[['posX', 'posY', 'posZ']]
    u, v = fit_line(points)
    u_standard, v_standard = standard_parametric_vector(u, v)
    distances = get_distances(u, v, points)
    track_statistic = {'eventID'    : track['eventID'].iloc[0],
                       'parentID'   : track['parentID'].iloc[0],
                       'PDGEncoding': track['PDGEncoding'].iloc[0],
                       'edep'       : track['edep'].to_list(),
                       'mean'       : distances.mean(),
                       'std'        : distances.std(),
                       'var'        : distances.var(),
                       'u'          : list(u_standard),
                       'v'          : list(v_standard)}
    statistic.append(track_statistic)

df_statistic = pd.DataFrame(statistic,
                            columns=['eventID', 'parentID', 'PDGEncoding', 'edep', 'mean', 'std', 'var', 'u', 'v'])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_statistic)
